In [1]:
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [2]:
# TODO

# change the date and period to an int (either month and year as seperate columns or a timestamp)
# experiment with column combinations more
# experiment with training on whole dataset

In [3]:
df = pd.read_csv('/Users/dhruv/code/export_yardi_jourentline.csv', lineterminator='\n')
df.head()

,GLCODE,GLNAME,PROPERTY,PROPERTYNAME,UNIT,BUILDING,SPECIALCIRCUMSTANCE,USAGEAMOUNT,USAGETYPE,DATE,PERIOD,DESCRIPTION,CONTROL,REFERENCE,AMOUNT,DEBITCREDIT,BALANCE,REMARKS
0,1110-0000,Cash,ap-mc11,4203 11TH LLC,42031,,,,,09/09/2020,12/01/2020,Webster S8 (t0034597),K-396581,165,8500.0,Credit,1813181.59,First buyout check
1,1110-0000,Cash,ap-mc11,4203 11TH LLC,42032,,,,,09/29/2020,12/01/2020,Webster (t0034598),K-405215,173,2500.0,Credit,1810681.59,
2,1110-0000,Cash,ap-mc11,4203 11TH LLC,42031,,,,,10/27/2020,12/01/2020,Webster S8 (t0034597),K-417819,183,9100.0,Credit,1801581.59,
3,1110-0000,Cash,ap-mc11,4203 11TH LLC,42052,,,,,10/29/2020,12/01/2020,Jones (t0034602),K-420897,187,3500.0,Credit,1798081.59,First Buyout Check
4,1110-0000,Cash,ap-mc11,4203 11TH LLC,42053,,,,,11/02/2020,12/01/2020,Dow (t0034603),K-421957,190,3500.0,Credit,1794581.59,First Buyout Check


In [4]:
# ***** PREPROCESSING *****

In [5]:
df.columns = df.columns.map(lambda c: c.strip())

cols = ['GLCODE', 'PROPERTY', 'UNIT', 'DATE', 'PERIOD', 'AMOUNT', 'DEBITCREDIT', 'DESCRIPTION', 'REMARKS']
df = df[cols]

df['DATE'] = df['DATE'].map(lambda d: datetime.strptime(d, ' %m/%d/%Y'))

df['DATEDAY'] = df['DATE'].map(lambda d: int(d.day))
df['DATEMO'] = df['DATE'].map(lambda d: int(d.month))
df['DATEYR'] = df['DATE'].map(lambda d: int(d.year))

df = df.drop(columns=['DATE'], axis=1)

df['REMARKS'] = df['REMARKS'].map(lambda r: r.lower())

df.head()


,GLCODE,PROPERTY,UNIT,PERIOD,AMOUNT,DEBITCREDIT,DESCRIPTION,REMARKS,DATEDAY,DATEMO,DATEYR
0,1110-0000,ap-mc11,42031,12/01/2020,8500.0,Credit,Webster S8 (t0034597),first buyout check,9,9,2020
1,1110-0000,ap-mc11,42032,12/01/2020,2500.0,Credit,Webster (t0034598),,29,9,2020
2,1110-0000,ap-mc11,42031,12/01/2020,9100.0,Credit,Webster S8 (t0034597),,27,10,2020
3,1110-0000,ap-mc11,42052,12/01/2020,3500.0,Credit,Jones (t0034602),first buyout check,29,10,2020
4,1110-0000,ap-mc11,42053,12/01/2020,3500.0,Credit,Dow (t0034603),first buyout check,2,11,2020


In [6]:
#glcodes = ['5305-0000', '5310-0000', '5315-0000']

#df['GLCODE'] = df['GLCODE'].map(lambda g: g if glcodes.count(g) > 0 else np.nan)

#df = df.dropna(axis=0)
#df = df.reset_index(drop=True)

#df.groupby('GLCODE').GLCODE.count()


In [7]:
X = df.copy()
X = X.drop(['AMOUNT'], axis=1)

y = df['AMOUNT']

preXTrain, preXTest, yTrain, yTest = train_test_split(X, y, test_size=0.15, random_state=1)


preXTrain = preXTrain.reset_index(drop=True)
yTrain = yTrain.reset_index(drop=True)
preXTest = preXTest.reset_index(drop=True)
yTest = yTest.reset_index(drop=True)


In [8]:
catCols = [col for col in X.columns if X[col].dtype == 'object']


In [9]:
labelXTrain = preXTrain.copy()
labelXTest = preXTest.copy()

le = LabelEncoder()

for col in catCols:
    le.fit(labelXTrain[col])
    
    labelXTest[col] = labelXTest[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
    
    le.classes_ = np.append(le.classes_, '<unknown>')
    
    labelXTrain[col] = le.transform(labelXTrain[col])
    labelXTest[col] = le.transform(labelXTest[col])
    
XTrain = labelXTrain.copy()
XTest = labelXTest.copy()


In [10]:
# ***** MODEL CREATION *****

In [11]:
model = RandomForestRegressor(n_estimators=40, random_state=0)
model.fit(XTrain, yTrain)

testPreds = pd.Series(model.predict(XTest))
testPreds = testPreds.map(lambda pred: round(pred, 2))

In [12]:
# ***** MODEL EVALUATION *****

In [13]:
acceptablePercent = 5  
percentsList = []

predHighCount = 0
predLowCount = 0

for i in range(0, len(yTest)):
    
    real = yTest[i]
    pred = testPreds[i]
    
    percentOfReal = (pred/real)*100
    percentOff = abs(100-percentOfReal)
    
    percentsList.append(round(percentOff, 2))
    
    
    if pred > real:
        predHighCount = predHighCount+1
    else:
        predLowCount = predLowCount+1
    

goodPreds = [x for x in percentsList if x < acceptablePercent]
badPreds = [x for x in percentsList if x > acceptablePercent]

print(len(goodPreds), len(badPreds))
print(predHighCount, predLowCount)


4749 9765
7238 7297


In [14]:
percentsDf = pd.DataFrame(percentsList, columns=['percentOff'])
percentsDf.head()

def categorizePercents(p):
    if p>=0 and p<5:
        return 5
    elif p>=5 and p<10:
        return 10
    elif p>=10 and p<20:
        return 20
    elif p>=20 and p<50:
        return 50
    elif p>=50 and p<100:
        return 100
    elif p>=100 and p<200:
        return 200
    elif p>=200 and p<500:
        return 500
    elif p>=500 and p<1000:
        return 1000
    elif p>=1000:
        return 10000
    else:
        return 'unknown'


percentsDf['category'] = percentsDf.apply(lambda p: categorizePercents(p['percentOff']), axis=1)
percentsDf.head()


,percentOff,category
0,12.65,20
1,33.98,50
2,6583.66,10000
3,3.25,5
4,1.93,5


In [15]:
percentRangeDf = percentsDf.groupby('category').size().reset_index(name='count')
percentRangeDf['percentOfTotal'] = percentRangeDf['count']/len(percentsDf.index)*100
print(percentRangeDf)


   category  count  percentOfTotal
0         5   4749       32.672859
1        10    987        6.790506
2        20   1236        8.503612
3        50   2032       13.980048
4       100   1813       12.473340
5       200    707        4.864121
6       500    697        4.795322
7      1000    420        2.889577
8     10000   1894       13.030616


In [16]:
comparison = pd.concat([yTest, testPreds], axis=1)
comparison.columns = ['actual', 'predicted']
print(comparison.iloc[0:25])


     actual  predicted
0    176.07     153.79
1    174.03     114.90
2      3.61     241.28
3    863.00     834.94
4    640.66     653.04
5    161.80     551.22
6    805.32     530.99
7      3.61      14.78
8   1602.00    1475.10
9   1800.00    1800.00
10     3.61       3.61
11  1365.00    1378.50
12     3.60       3.82
13  1091.22     478.22
14     3.61       3.61
15     3.61     583.88
16    90.00      91.88
17    70.00     373.76
18   836.41     283.13
19    97.60     137.83
20     3.60       3.75
21    89.15      86.28
22     6.31     235.40
23     1.34     528.13
24    33.06     522.09


In [17]:
print('R2:', model.score(XTest, yTest))
print('MAE:', metrics.mean_absolute_error(yTest, testPreds))
print('RMSE:', metrics.mean_squared_error(yTest, testPreds, squared=False))


R2: 0.5284761647693281
MAE: 1706.3541087031306
RMSE: 31671.138018247442
